In [1]:
import scanpy as sc
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, LinearColorMapper,ColorBar, LinearAxis, Range1d,LinearScale
from bokeh.models.transforms import CustomJSTransform
from bokeh.transform import factor_cmap, transform
import sys
sys.path.append('../beehive/')
os.environ['BEEHIVE_BASEDIR'] = '/Users/raghidbsat/Extras_No_Icloud_save/VIB/bds_lab/beehive'
import os
import numpy as np
import beehive
from beehive import config, util, expset
from bokeh.palettes import Category20
import polars as pl
from bokeh.util.hex import cartesian_to_axial
from bokeh.transform import jitter, CategoricalColorMapper


In [2]:
output_notebook()

Loading BokehJS ...

In [14]:
meta1 = "oAbvsScr"
meta2 = "cell.state"
meta3 = "mouse.id"
gene = "APOE"
genedata = expset.get_gene("h.man2m.1.sct",gene)
metadata = expset.get_meta("h.man2m.1.sct",meta1,view_name = "gene_expression", raw = True) #facet1
metadata2 = expset.get_meta("h.man2m.1.sct",meta2,view_name = "gene_expression", raw = True) #facet2
metadata3 = expset.get_meta("h.man2m.1.sct",meta3,view_name = "gene_expression", raw = True) #most likely mouse_id
    

metadata = metadata.select((pl.col(meta1)).cast(str))
metadata2 = metadata2.select((pl.col(meta2)).cast(str))
metadata3 = metadata3.select((pl.col(meta3)).cast(str))


if genedata.columns == metadata.columns:
    metadata.columns = [metadata.columns[0] + "_category"]
if genedata.columns == metadata2.columns:
    metadata2.columns = [metadata2.columns[0] + "_category"]
if genedata.columns == metadata3.columns:
    metadata3.columns = [metadata3.columns[0] + "_category"]

new_meta = metadata.columns[0]
new_meta2 = metadata2.columns[0]
new_meta3 = metadata3.columns[0]

metadata4 = pl.DataFrame([])

rv_combined = (
    pl.concat([genedata, metadata,metadata2,metadata3], how="horizontal")
    .groupby([new_meta,new_meta2])
    .agg(
        [
            pl.count(),
            pl.mean(gene).alias("mean"),
            pl.std(gene).alias("std"),
            pl.median(gene).alias("median"),
            pl.quantile(gene, 0.25).alias("q25"),
            pl.quantile(gene, 0.75).alias("q75"),
            pl.quantile(gene, 0.01).alias("q01"),
            pl.quantile(gene, 0.99).alias("q99"),
            pl.min(gene).alias("min"),
            pl.max(gene).alias("max"),
        ]
    )
)


rv_combined

oAbvsScr,cell.state,count,mean,std,median,q25,q75,q01,q99,min,max
str,str,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Scr""","""Antigen-presen...",461,2.545145,1.372957,2.833213,1.791759,3.610918,0.0,4.5951,0.0,5.252273
"""Scr""","""Homeostatic (H...",6224,2.108547,1.133393,2.302585,1.386294,2.995732,0.0,4.2626,0.0,5.0937
"""Scr""","""Cytokines resp...",89,2.668108,1.002409,2.890372,2.079442,3.258097,0.0,4.8978,0.0,4.8978
"""Scr""","""Disease associ...",725,3.318719,0.882683,3.465736,2.833213,3.912023,0.693147,4.882802,0.0,5.337538
"""NONE""","""Cytokines resp...",2741,3.673143,0.825925,3.806662,3.258097,4.234107,0.693147,5.105945,0.0,5.768321
"""oAb""","""Cytokines resp...",3828,1.902911,1.064326,2.079442,1.098612,2.708,0.0,3.951244,0.0,5.209486
"""Scr""","""Interferon res...",447,2.562532,0.907817,2.708,2.079442,3.258097,0.0,4.158883,0.0,4.905275
"""oAb""","""Homeostatic (H...",6271,2.2044,1.132807,2.397895,1.386294,3.044522,0.0,4.2626,0.0,5.068904
"""NONE""","""Cytokines resp...",5522,3.589855,0.9259,3.7612,3.178054,4.204693,0.0,5.0937,0.0,5.645447


In [53]:
datadir = util.get_datadir("h5ad")
dsid ='h.s423a.1'
#print(expset.get_varfields(dsid))
# rv2 = pl.read_parquet(datadir / f"{dsid}.obs.prq", ["slice_id"] + ["X.in.pixel"] + ["Y.in.pixel"] + ["__index_level_0__"])
# rv1 = pl.read_parquet(datadir / f"{dsid}.X.prq")
# rv.to_pandas()


abundance = pl.read_parquet(datadir / f"{dsid}.var.prq", ["diffvsnon_bin__res__lcpm"])
meanexp = pl.read_parquet(datadir /f"{dsid}.var.prq", ["diffvsnon_bin__res__lfc"] )
rv = pl.concat([abundance,meanexp], how="horizontal")
data = rv.to_pandas()
data.dropna(axis = 0, inplace = True)
source = ColumnDataSource(data)
plot = figure()

plot.scatter(x="diffvsnon_bin__res__lcpm",y="diffvsnon_bin__res__lfc", source = source)
show(plot)


In [ ]:
#spot ID is __index_level_0__ in rv
# x,y coordinates in .obs and all gene values related
import csv
import json
 
 
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column named 'No' to
            # be the primary key
            key = rows['__index_level_0__']
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         
# Driver Code
 
# Decide the two file paths according to your
# computer system
csvFilePath = "h.s423a.1.obs.csv"
jsonFilePath = "h.s423a.1.obs.json"

# Call the make_json function
make_json(csvFilePath, jsonFilePath)

: 

: 

In [ ]:

datadir = util.get_datadir("h5ad")
dsid ='h.man2m.1.sct'
#print(expset.get_varfields(dsid))
rv = pl.read_parquet(datadir / f"{dsid}.X.prq")
# rv.to_pandas()

In [147]:
gene = "APOE"
meta2 = "oAbvsScr"
meta = "cell.state"
genedata = expset.get_gene(dsid, gene)
metadata = expset.get_meta(dsid, meta)
metadata2 = expset.get_meta(dsid,meta2,raw=True)
mouseids = expset.get_meta(dsid,"mouse.id")


rv_combined = (
        pl.concat([genedata, metadata,metadata2,mouseids], how="horizontal")
        .groupby([meta,meta2])
        .agg(
            [
                pl.count(),
                pl.mean(gene).alias("mean"),
                pl.std(gene).alias("std"),
                pl.median(gene).alias("median"),
                pl.quantile(gene, 0.25).alias("q25"),
                pl.quantile(gene, 0.75).alias("q75"),
                pl.quantile(gene, 0.01).alias("q01"),
                pl.quantile(gene, 0.99).alias("q99"),
                pl.min(gene).alias("min"),
                pl.max(gene).alias("max"),
            ]
        )
    )


rv_mouseid = (
        pl.concat([genedata, metadata,metadata2,mouseids], how="horizontal")
        .groupby([meta,meta2,"mouse.id"])
        .agg(
            [
                pl.count().alias("count_mouseid"),
                pl.mean(gene).alias("mean_mouseid"),
            ]
        )
    )

rv_combined = rv_combined.to_pandas()
rv_mouseid = rv_mouseid.to_pandas()


rv_combined['perc'] = 100 * rv_combined['count'] / rv_combined['count'].sum()
rv_combined['_segment_top'] = rv_combined['q99']
rv_combined['_bar_top'] = rv_combined['q75']
rv_combined['_bar_median'] = rv_combined['median']
rv_combined['_bar_bottom'] = rv_combined['q25']
rv_combined['_segment_bottom'] = rv_combined['q01']


x = [ (genotype, cellstate) for genotype in rv_combined[meta2].tolist() for cellstate in rv_combined[meta].tolist() ]
# rv_combined["x"] = list(set(x))
# rv_combined["x"] = sorted(rv_combined["x"],key=lambda tup: tup[0])

rv_combined["x"] = rv_combined[[meta2,meta]].apply(tuple, axis=1)
rv_combined["x"] = sorted(rv_combined["x"],key=lambda tup: tup[0])
rv_combined["x"] = sorted(rv_combined["x"],key=lambda tup: tup[1])


rv_mouseid["x"] = rv_mouseid[[meta2,meta]].apply(tuple, axis=1)
rv_mouseid["x"] = sorted(rv_mouseid["x"],key=lambda tup: tup[0])
rv_mouseid["x"] = sorted(rv_mouseid["x"],key=lambda tup: tup[1])


final_rv = pd.merge(rv_combined,rv_mouseid,on="x")

final_rv["x"] = sorted(final_rv["x"],key=lambda tup: tup[0])
final_rv["x"] = sorted(final_rv["x"],key=lambda tup: tup[1])


# final_rv = final_rv.drop_duplicates("x")
final_rv["cell.state_y"] = sorted(final_rv["cell.state_y"])


source2 = ColumnDataSource(final_rv)
plot2 = figure(x_range = [], width  = 1000, height = 500)
plot2.x_range.factors = sorted(rv_combined["x"].to_list(),key=lambda tup: tup[0])
# plot2.x_range.factors = sorted(rv_combined["x"].to_list(),key=lambda tup: tup[1])

dict_colors = {"Homeostatic (HM)": "#E8847B", "Transitioning CRM" : "#E29874",
"Cytokines response 1 (CRM-1)" : "#B6854D",
 "Cytokines response 2 (CRM-2)" : "#D3AE74", 
"Ribosomal response (RM)" : "#788AA3",
"Disease associated (DAM)" : "#729489",
"Antigen-presenting response (HLA)" : "#6FC3B9",
"Interferon response (IRM)" : "#9A6884"}

mapper = CategoricalColorMapper(palette = list(dict_colors.values()), factors=list(dict_colors.keys()))

vbar=plot2.vbar(x="x", top='_bar_top',
                bottom='_bar_bottom', source = source2, width=0.85, name="barplot",
                fill_color={'field': 'cell.state_y', 'transform': mapper},
                line_color="black")
jitter_points = plot2.scatter(x=jitter('x', width=0.4, range=plot2.x_range), y='mean_mouseid', size=5, alpha=0.4, source=source2)


# seg_v_up=plot2.segment(source=source2, x0='x', x1='x',
#                         y0='_bar_top', y1='_segment_top',
#                         line_color='black'),
# seg_h_up=plot2.rect(source=source2, x='x', height=0.001,
#                     y='_segment_top', width=0.4, line_color='black'),
# seg_v_dw=plot2.segment(source=source2, x0='x', x1='x',
#                         y0='_segment_bottom', y1='_bar_bottom',
#                         line_color='black'),
# seg_h_dw=plot2.rect(source=source2, x='x', height=0.001,
#                     y='_segment_bottom', width=0.4, line_color='black'),
# seg_h_med=plot2.rect(source=source2, x='x', height=0.001,
#                     y='_bar_median', width=0.85, line_width=2,
#                     line_color='black'),

plot2.xaxis.major_label_orientation = 3.14/2
        
show(plot2)


In [11]:
gene = "APOE"
meta = "cell.state"
meta2 = "oAbvsScr"
meta3 = "mouse.id"
dsid ='h.man2m.1.sct'
final_rv,factors = expset.get_gene_meta_three_facets(dsid,gene,meta,meta2,meta3)
source = ColumnDataSource(final_rv)


###plot###
plot = figure(x_range = [], width  = 1000, height = 500)
plot.x_range.factors = sorted(list(set(factors)),key=lambda tup: tup[0])


vbar=plot.vbar(x="x", top='_bar_top',
                bottom='_bar_bottom', source = source, width=0.85, name="barplot",
                fill_color=config.colors.color6,
                line_color="black"),
jitter_points = plot.scatter(x=jitter('x', width=0.4, range=plot.x_range), y=f'mean_{meta3}', size=5, alpha=0.4, source=source)
plot.xaxis.major_label_orientation = 3.14/2
show(plot)

In [189]:
from bokeh.util.hex import cartesian_to_axial

###get Data####
# data = pd.DataFrame(dict(
#         gene1 = expset.get_gene(dataset_id, "APOE")[:,0],
#         gene2 = expset.get_gene(dataset_id, "TREM2")[:,0],
#         obs = expset.get_meta(dataset_id, "cell.state")[:,0],
#         gene3 = expset.get_gene(dataset_id, "APOC1")[:,0],
#         ))

dataset_id = "h.man2m.1.sct"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))


#manipulate data to get axial coordinates
q, r = cartesian_to_axial(data["gene1"], data["gene2"], 0.1, "pointytop")
df = pd.DataFrame(dict(r=r,q=q,avg_exp = None))
groups = df.groupby(["q","r"])
dicts = []
###assign axial coordinates to get average gene expression###
for key,val in groups.groups.items():
    q,r = key
    counts = len(val)
    avg_exp = data['gene3'].loc[groups.groups.get((q,r))].mean()
    dicts = dicts + [{"q" : q,"r" : r,"counts" : counts, "avg_exp" :avg_exp,"index_list":val}]
    

##use in source
final_result = pd.DataFrame(dicts)
source_bin = ColumnDataSource(final_result)



###mappers###
mapper = LinearColorMapper(
    palette='Magma256',
    low=final_result["avg_exp"].max(),
    high=final_result["avg_exp"].min())

v_func_alpha  = """
var new_xs = new Array(xs.length)
for(var i = 0; i < xs.length; i++) {
    new_xs[i] = alpha_map[xs[i]]
}
return new_xs
"""

###plot###
#make alpha optional => checkbox
#plot any continuous to any continuous
#subset for categorical fields => drop down (mutliselectfield)
#color of the third continious variable is based on the full data with the obs
#first layer is everything (with no colors one color only.) grey light
#second layer is the conitnuous variable colored + the possibility to subset
#have the option to either color or not color 
#widget for hex size
length = len(sorted(np.unique(source_bin.data["counts"]).tolist()))
alpha_map = dict(zip(sorted(np.unique(source_bin.data["counts"]).tolist()),[x/length for x in range(50,length)]))

numerical_alpha_transformer = CustomJSTransform(args={"alpha_map": alpha_map}, v_func=v_func_alpha)

# plot = figure()
# plot.hex_tile(q="q",r="r",size=0.1,source=source_bin,
# alpha = transform("counts",numerical_alpha_transformer),line_color = None, 
# color  = {'field': 'avg_exp', 'transform': mapper})


plot = figure()
plot.hex_tile(q="q",r="r",size=0.1,source=source_bin, 
color  = "#D3D3D3")

plot.hex_tile(q="q",r="r",size=0.1,source=source_bin,
alpha = transform("counts",numerical_alpha_transformer),line_color = None, 
color   = {'field': 'avg_exp', 'transform': mapper})

plot.xaxis.axis_label = "APOE"
plot.yaxis.axis_label = "TREM2"
color_bar1 = ColorBar(color_mapper=mapper, location=(0,0),major_label_text_font_size = "0px",major_tick_in = 2,title = "APOC1 average expression")
plot.add_layout(color_bar1,"right")
plot.extra_y_ranges = {"y2": Range1d(start = final_result["counts"].min(), end = final_result["counts"].max())}
extra_axis = LinearAxis(y_range_name = "y2",axis_label = "Counts by Transparency per Hex tile")
plot.add_layout(extra_axis, 'right')


show(plot)


In [193]:

###get Data####
# data = pd.DataFrame(dict(
#         gene1 = expset.get_gene(dataset_id, "APOE")[:,0],
#         gene2 = expset.get_gene(dataset_id, "TREM2")[:,0],
#         obs = expset.get_meta(dataset_id, "cell.state")[:,0],
#         gene3 = expset.get_gene(dataset_id, "APOC1")[:,0],
#         ))

dataset_id = "h.man2m.1.raw"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        # gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        gene2 = expset.get_gene(dataset_id, "MALAT1")[:,0],

        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))

#manipulate data to get axial coordinates
q, r = cartesian_to_axial(data["gene1"], data["gene2"], 0.1, "pointytop")
df = pd.DataFrame(dict(r=r,q=q,avg_exp = None))
groups = df.groupby(["q","r"])
dicts = []
###assign axial coordinates to get average gene expression###
for key,val in groups.groups.items():
    q,r = key
    counts = len(val)
    avg_exp = data['gene3'].loc[groups.groups.get((q,r))].mean()
    dicts = dicts + [{"q" : q,"r" : r,"counts" : counts, "avg_exp" :avg_exp,"index_list":val}]
    

##use in source
final_result = pd.DataFrame(dicts)
source_bin = ColumnDataSource(final_result)



###mappers###
mapper = LinearColorMapper(
    palette='Magma256',
    low=final_result["avg_exp"].max(),
    high=final_result["avg_exp"].min())

# v_func_alpha  = """
# var new_xs = new Array(xs.length)
# for(var i = 0; i < xs.length; i++) {
#     new_xs[i] = alpha_map[xs[i]]
# }
# return new_xs
# """

plot = figure()
# plot.hex_tile(q="q",r="r",size=0.1,source=source_bin, 
# color  = "#D3D3D3")

plot.hex_tile(q="q",r="r",size=0.1,source=source_bin,line_color = None, 
color   = {'field': 'avg_exp', 'transform': mapper},alpha = 1)



show(plot)


In [241]:
dataset_id = "h.man2m.1.raw"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        # gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        gene2 = expset.get_gene(dataset_id, "MALAT1")[:,0],

        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))

#manipulate data to get axial coordinates
max1 = max(data["gene2"]) - max(data["gene1"])
min1 = min(data["gene2"]) - min(data["gene1"])
q, r = cartesian_to_axial(data["gene1"], data["gene2"], len(data["gene2"])/max1, "pointytop")
df = pd.DataFrame(dict(r=r,q=q,avg_exp = None))

In [274]:
dataset_id = "h.man2m.1.sct"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        # gene2 = expset.get_gene(dataset_id, "MALAT1")[:,0],

        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))

#manipulate data to get axial coordinates
q, r = cartesian_to_axial(data["gene1"], data["gene2"], size = 0.1, orientation =  "pointytop")

df = pd.DataFrame(dict(r=r,q=q,avg_exp = None))

plot = figure(match_aspect = True)
plot.hex_tile(q="q",r="r",source=df,line_color = None,alpha = 1)
show(plot)